In [1]:
import pandas as pd
import numpy as np
import transformers
from transformers import (
    pipeline,
    TextClassificationPipeline,
    AutoTokenizer
)
import torch

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LEN = 512
# TRAIN_BATCH_SIZE = 32
# VALID_BATCH_SIZE = 32
# EPOCHS = 10
# LEARNING_RATE = 3e-05


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



cuda


In [10]:
data = pd.read_parquet("test.gzip")
data

,sentences,tags
0,Mix all together.|Add tiny marshmallows and re...,"[1, 0]"
1,Spread in a casserole dish.|Mix all ingredients.,"[0, 1]"
2,Toss and cut with fork until blended.|Save 1/4...,"[0, 1]"
3,Place in large kettle and cover with water to ...,"[0, 1]"
4,Makes 11 to 12 cups.|Real good.,"[1, 0]"
...,...,...
59995,"Beat egg.|Stir in milk, sugar and shortening.","[1, 0]"
59996,"Cream sugar, oleo and Crisco together. Add egg...","[1, 0]"
59997,Cover bottom.|Sprinkle cake mix over butter. S...,"[1, 0]"
59998,"If needed, add more liquid.|Fold in bananas.","[0, 1]"


In [15]:
data.iloc[56][1]


array([1, 0], dtype=int64)

In [2]:
model_path = "./bert-nsp-rand-nolora-output/"
model_c = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
# model_c.config.problem_type = "regression"
model_c.to(device)
model_c.num_parameters()


108311810

In [ ]:
class logitpipeline(TextClassificationPipeline):
    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"]
        return best_class
pipe = logitpipeline(model=model_c, tokenizer=tokenizer)

In [5]:
pipe2 = TextClassificationPipeline(model=model_c, tokenizer=tokenizer)

In [8]:
pipe2({'text':"sentence 1", "text_pair":"sentence 2"})

{'label': 'LABEL_1', 'score': 0.7315961122512817}

In [9]:
# def handle_pair(pair):
#     encoding = tokenizer.encode_plus(pair[0],pair[1],
#                                      add_special_tokens=True,
#                                     max_length=512,
#                                     padding='max_length',
#                                     return_token_type_ids=True,
#                                     truncation=True)
#     ret = [torch.tensor(val, dtype=torch.long).to("cuda") for val in encoding.values()]
#     return ret
# count = 0
# def pair_to_bool(sent):
#     global count
#     count += 1
#     if count%100==0:
#         print(count, end='\r')
#     tokens = handle_pair(sent)
#     # print([len(i) for i in tokens])
#     model_c.eval()
#     with torch.no_grad():
#         predict = model_c(input_ids=tokens[0].unsqueeze(0),attention_mask=tokens[1].unsqueeze(0),token_type_ids=tokens[2].unsqueeze(0))
#     return predict.logits

In [10]:
ground = [np.argmax(i) for i in data.tags.tolist()]
sents = [i.split("|") for i in data.sentences.tolist()]

In [12]:
def data():
    for i in range(len(sents)):
        yield {'text':sents[i][0], 'text_pair':sents[i][1]}
output = []
count = 0
for out in pipe(data()):
    count += 1
    if count % 100 ==0:
        print(count, end='\r')
    output.append(out)

In [19]:
out_df = pd.DataFrame({"output":[[float(a) for a in i[0]] for i in output]})

In [21]:
out_df.to_parquet("pred_rand_nolora.gzip", compression='gzip')

In [20]:
out_df

,output
0,"[0.9197629690170288, -0.9144289493560791]"
1,"[-4.820921421051025, 4.817933082580566]"
2,"[0.9184244275093079, -0.9286680221557617]"
3,"[-5.570468902587891, 5.565320014953613]"
4,"[1.2122944593429565, -1.2354772090911865]"
...,...
59995,"[1.8639991283416748, -1.8530266284942627]"
59996,"[-0.02368032932281494, 0.02230800874531269]"
59997,"[0.8871720433235168, -0.8760192394256592]"
59998,"[1.516648292541504, -1.5160162448883057]"


In [22]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Hello world."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [26]:
(output[1].shape)

torch.Size([1, 768])

In [25]:
# encodings = tokenizer.batch_encode_plus(
#                     data['sentences'].tolist(),
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )

In [7]:
# # def enc(sent)
# encodings = tokenizer.encode_plus(
#                     data['sentences'].tolist()[0],
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )
# ret = [val.clone().detach().to("cuda") for val in encodings.values()]
# ret

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
# class cooking_time_model(torch.nn.Module):
#     def __init__(self):
#         super(cooking_time_model, self).__init__()
#         self.l1 = transformers.RobertaModel.from_pretrained(pretrained)
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, MAX_LEN)

#     def forward(self, ids, mask, token_type_ids):
#         output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
#         output_2 = self.l2(output_1.pooler_output)
#         output = self.l3(output_2)
#         return output
model_path = "./drob-mfst-nolora-output/checkpoint-17500"
model_c = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=1, load_in_8bit = True)
model_c.config.problem_type = "regression"
# model_c = cooking_time_model()
# model_c.to(device)
# def count_parameters(modelr):
#     return sum(p.numel() for p in modelr.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model_c):,} trainable parameters')

In [38]:
len(encodings)

3

In [6]:
model_c.eval()
with torch.no_grad():
    output = model_c(input_ids=ret[0],attention_mask=ret[1],token_type_ids=ret[2])

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [22]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [23]:
model_best, _, _, l1 = load_ckp("best_model_a3.pt", model_c, optimizer)
model_current, _, _, l2 = load_ckp("current_checkpoint_a3.pt", model_c, optimizer)
print(l1==l2)
print(l1, l2)

True
0.00022455735384061483 0.00022455735384061483


In [11]:
def submission(dataloader, model_selected): # Code for submission
    preds = []
    targets = []
    model_selected.eval()
    with torch.no_grad():
        for batch_idx, (id, mask, tokentype, target) in enumerate(dataloader):
            outputs = model_selected(id, mask, tokentype)
            preds.append(list(outputs.logits))
            targets.append(list(target))
            if (batch_idx+1)%50 == 0:
                print("~"*15,f"{batch_idx+1}/{len(dataloader)}({(batch_idx+1)/len(dataloader)})","~"*15, end="\r")
            
    return preds, targets
ret, tar = submission(test_loader, model_best)

In [12]:
ltar = [l[0].tolist() for l in tar]
lret = [l[0].tolist()[0] for l in ret]
ltar = [round(l, 4) for l in ltar]
lret = [round(l, 4) if l>=0 else 0 for l in lret]
# lret = [l[0] for l in ret]

In [13]:
# for p, r in zip(ltar, lret):
#     # print(p, r)
#     print(abs(p-r)*100)
err = [abs(p-r) for p,r in zip(ltar,lret)]
print(np.mean(err))

0.006316959055973027


In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
err_min = scaler.inverse_transform([[i] for i in err])
# for m, e in zip(err_min, err):
#     print(m[0], e)
print("error stats:")
print("average:", np.mean(err_min))
print("max:", *err_min[np.argmax(err_min)])
print("max error target(minutes)",(scaler.inverse_transform([[data.iloc[np.argmax(err_min)][1]]])[0][0]))
print("max error prediction(minutes)",scaler.inverse_transform([[lret[np.argmax(err_min)]]])[0][0])
print("max error recipe:")
print(data.iloc[np.argmax(err_min)][0])
# data


error stats:
average: 0.6316959055973027
max: 92.7
max error target(minutes) 93.5
max error prediction(minutes) 0.8
max error recipe:
1 elephant, 2 rabbits (optional), 2 pails salt, 1 (8 oz.) canister black pepper, 3 bushels onions, <mask> gal. water, 6 pails flour


In [40]:
# test_input = ["<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbs. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chilli flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped"]
test_input = ['6 slices bacon, chopped, 0.5 c. chopped onion, <mask> c. chopped celery, 0.5 c. chopped green bell pepper, 3 cloves garlic, minced, 0.5 tsp. kosher salt, 0.5 tsp. freshly ground black pepper, 0.5 tsp. dried thyme, 0.25 tsp. cayenne pepper, 2 (12 ounce) packages frozen black-eyed peas, 4 c. reduced-sodium chicken broth, 2 c. cooked long grain rice, 0.25 c. chopped green onions, or as needed']
# test_input = ['<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbsp. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chili flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped']
test_input = ["0.25 c. unsalted butter, 0.25 c. all-purpose flour, 1 medium onion, chopped, 1 c. chopped celery, 0.67 c. chopped red bell pepper, 0.67 c. chopped green bell pepper, 2 cloves garlic, minced, 2 tsp. Cajun seasoning, 0.5 c. chicken broth, 0.5 lb. thinly sliced, fully cooked smoked sausage, 0.25 tsp. crushed red pepper flakes, 0.5 tsp. freshly ground black pepper, salt to taste (see Cook's Notes), 0.5 lb. raw shrimp, peeled and deveined (see Cook's Notes), 0.5 c. heavy whipping cream, <mask> oz. spaghetti, cooked, chopped fresh parsley for garnish (optional), lemon slices for garnish (optional)"]
def test_pipeline(test_input):
    single_df = pd.DataFrame({"ingr":test_input, "tags":[0.0]})
    # single_df
    s_test_customset = CustomDataset(single_df, tokenizer, MAX_LEN)
    s_test_loader = DataLoader(s_test_customset, batch_size=1, shuffle=False, collate_fn=colbatch)
    with torch.no_grad():
        for _, (id, mask, tokentype, _) in enumerate(s_test_loader):
            outputs = model_best(id, mask, tokentype)
            print(outputs.logits.tolist()[0][0])
            print(scaler.inverse_transform(outputs.logits.tolist())[0][0])
            return(outputs.logits.tolist()[0][0])
            
            # return outputs.logits.tolist(), scaler.inverse_transform(outputs.logits.tolist())
test_pipeline(test_input)
scaler.transform([[12]])

0.08980672806501389
8.980672806501389


array([[0.12]])